<a href="https://colab.research.google.com/github/brunogresende/ppca-aed/blob/main/Aloca%C3%A7%C3%A3o_de_Pessoas_em_Trabalho_Remoto_Scheduling_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
#!pip install -U pulp
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, LpConstraint
import math

In [158]:
#notations
n     = 20  #number of days
#tAJr  = 5  #number of junior analysts
#tASr  = 1  #number of senior analysts
#tE    = 1  #humber of specialists
#max_absence = int((tAJr + tASr + tE)*0.3) #max absence allowed in team
t     = 50 #number of employees
max_day_absence = math.floor(t*0.3) #max absence allowed in team

#decision variables
var_L   = LpVariable.dicts("Absence", (range(n), range(t)), 0, 1, "Binary")
var_E   = LpVariable.dicts("Remote", (range(n), range(t)), 0, 1, "Binary")
#var_L   = LpVariable.dicts("Leave", (range(n), range(tAJr + tASr + tE)), 0, 1, "Binary")
#var_E   = LpVariable.dicts("HomeEsp", (range(n), range(tE)), 0, 1, "Binary")
#var_AJr = LpVariable.dicts("HomeAJr", (range(n), range(tAJr)), 0, 1, "Binary")
#var_ASr = LpVariable.dicts("HomeASr", (range(n), range(tASr)), 0, 1, "Binary")

#objective function
obj = None
for i in range(n):
  for j in range(t):
    obj += var_E[i][j] #+ var_L[i][j]

#for i in range(n):
#  for j in range(tE):
#    obj += var_E[i][j]
#    for jj in range(tAJr):
#      obj += var_AJr[i][jj]
#      for jjj in range(tASr):
#        obj += var_ASr[i][jjj]

problem = LpProblem("Schedule", LpMaximize)
problem += obj


In [159]:
max_day_absence

15

In [160]:
#constraint 0
#absences configuration

#funci 5, 0-5 dias de férias
for i in range(0,5):
  problem += var_L[i][5] == 1

#funci 3, 5-15 dias de férias
for i in range(5,15):
  problem += var_L[i][3] == 1

#funci 7, 3 dias de abono
for i in range(17,19):
  problem += var_L[i][7] == 1

#funci 8, 2 dias de abono
for i in range(4,6):
  problem += var_L[i][8] == 1


In [161]:
#constraint 1
#cada funcionário poderá estar trabalhando remoto (1), trabalhando no escritório (0) ou ausente (1), de forma excludente. 
c = None
for i in range(n):
  for j in range(t):
    c = lpSum(var_E[i][j] + var_L[i][j]) <= 1
    problem += c  
        
#constraint 2 
#cada funcionário poderá ser alocado no máximo 8 dias em home-office no mês. 
for j in range(t):
  c = None
  for i in range(n):
    c += var_E[i][j]
  problem += c <=8

#constraint 3
#a quantidade máxima de ausências do escritório em um dia deve ser no máximo 30% do total de funcionários da equipe. 
c = None
for i in range(n):
  for j in range(t):
    c += var_E[i][j] + var_L[i][j]
problem += (c <= max_day_absence*n)

In [162]:
# See the modeled problem
#problem

In [163]:
# Solve the problem
status = problem.solve()

# print results
print(f"Solution Time: {problem.solutionTime}")
print(f"Solution CPU Time: {problem.solutionCpuTime}")
print(f"Status: {problem.status}, {LpStatus[problem.status]}")
print(f"Objective: {problem.objective.value()}")

#for var in problem.variables():
#    print(f"{var.name}: {var.value()}")

#for name, constraint in problem.constraints.items():
#    print(f"{name}: {constraint.value()}")

Solution Time: 0.07715296745300293
Solution CPU Time: 0.07800000000133878
Status: 1, Optimal
Objective: 281.0
